conda create -n geo python

conda activate geo

conda install -c conda-forge mamba

mamba install -c conda-forge segment-geospatial

pip install ultralytics

In [54]:
# import samgeo
import os
import torch
# from samgeo import SamGeo, tms_to_geotiff
from tms_to_geotiff import tms_to_geotiff

In [ ]:

# bottom left, top right,
bbox = [98.97833929776746,18.782291559223832,  98.99318992500658, 18.794283998891945]
image = 'satellite.tif'
tms_to_geotiff(output=image, bbox=bbox, zoom=20, source='Satellite', overwrite=True)

In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
# Load a pretrained YOLOv8n model
model = YOLO("best_joam.pt")

# Run inference on 'bus.jpg' with arguments
results = model.predict("satellite.tif", save=True, imgsz=640, conf=0.5)
# results = bestModel.predict(source=imagePath, imgsz=640)

annotatedImage = results[0].plot()
annotatedImageRGB = cv2.cvtColor(annotatedImage, cv2.COLOR_BGR2RGB)
plt.imshow(annotatedImageRGB)
plt.axis('off')
plt.tight_layout()
plt.show()

In [57]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import rasterio
from rasterio.features import shapes
from shapely.geometry import shape, mapping
import geopandas as gpd
import pandas as pd
import numpy as np
import os

model = YOLO("best_joam.pt")

results = model.predict("satellite.tif", save=True, imgsz=640, conf=0.5)

with rasterio.open("satellite.tif") as src:
    transform = src.transform
    crs = src.crs
    height, width = src.height, src.width

gdfs = []

for i, mask in enumerate(results[0].masks.data):
    mask = mask.cpu().numpy().astype('uint8')
    
    class_index = int(results[0].boxes.cls[i])
    label = results[0].names[class_index]

    output_file = f"segment_{i+1}.tif"
    with rasterio.open(
        output_file,
        "w",
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype=mask.dtype,
        crs=crs,
        transform=transform,
    ) as dst:
        dst.write(mask, 1)
    
    with rasterio.open(output_file) as src:
        mask_data = src.read(1)
        mask_transform = src.transform

        shapes_gen = shapes(mask_data, transform=mask_transform)
        polygons = [shape(geom) for geom, val in shapes_gen if val == 1]

        gdf = gpd.GeoDataFrame(geometry=polygons, crs=crs)
        gdf['label'] = label
        gdfs.append(gdf)

    os.remove(output_file)

combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))
combined_geojson_output_file = "combined_segments.geojson"
combined_gdf.to_file(combined_geojson_output_file, driver='GeoJSON')

print(f"Saved combined GeoJSON to {combined_geojson_output_file}")





image 1/1 /Users/sakdahomhuan/Dev/segment-geospatial/tests/satellite.tif: 320x640 8 Cement Roofs, 18 Metal roofs, 1 Other roof, 19 Concrete roofs, 2184.6ms
Speed: 8.0ms preprocess, 2184.6ms inference, 29.2ms postprocess per image at shape (1, 3, 320, 640)
Results saved to /opt/homebrew/runs/segment/predict20
Saved combined GeoJSON to combined_segments.geojson
